In [1]:
import sys
sys.path.append("../../Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages")

In [4]:
class Dataset:
    def __init__(self, data:dict):
        self._data=data
        self._original=data
    def reset_data(self):
        self.data=self.original
    def keys(self):
        return self._data.keys()
    def values(self):
        return self._data.values()
    def items(self):
        return self._data.items()

In [7]:
strategy_labels={0:"random",1:"single_word_syn",2:"n_word_syn",3:"single_word_ant",4:"n_word_ant",5:"misspell",6:"reorder",7:"addition",8:"deletion"}
strategy_ids={"random":0,"single_word_syn":1,"n_word_syn":2,"single_word_ant":3,"n_word_ant":4,"misspell":5,"reorder":6,"addition":7,"deletion":8}

In [8]:
class MutagenDataset(Dataset):
    def __init__(self, data:dict, mutations=[], strategy="random"):
        super().__init__(self, data)
        self._mutations=mutations
        self._strategy=strategy
        self._stratid=strategy_ids[strategy]
    def reset_data(self):
        super().reset_data()
        self._mutations=[]
    def appendMutations(self,mutation:list):
        self._mutations.append(mutation)